Merging the 3 CSV files from QS, THE, SH rankings

In [2]:
!pip install fuzzywuzzy
!pip install python-Levenshtein

DEPRECATION: amazon-textract-pipeline-pagedimensions 0.0.8 has a non-standard dependency specifier Pillow>=9.4.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of amazon-textract-pipeline-pagedimensions or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: amazon-textract-pipeline-pagedimensions 0.0.8 has a non-standard dependency specifier pypdf>=2.5.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of amazon-textract-pipeline-pagedimensions or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: amazon-textract-pipeline-pagedimensions 0.0.8 has a non-standard dependency specifier Pillow>=9.4.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of amazon-textract-pipeline-pagedimensions or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: amazon-textract-pipeline-pagedimensions 0.0.8 has a non-standard dependency specifier pypdf>=2.5.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of amazon-textract-pipeline-pagedimensions or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:
# import pandas library
import pandas as pd

# change csv file names as necessary
df_qs = pd.read_csv('qs_csv.csv')
df_the = pd.read_csv('the_csv.csv')
df_shanghai = pd.read_csv('sh_csv.csv')

In [4]:
# QS uni name cleaning
df_qs['University'] = df_qs['University'].str.lower().replace('\s\(.*\)', '', regex=True)
df_qs['University'] = df_qs['University'].str.lower().replace('[^a-z0-9 ]', '', regex=True)
df_qs

,Unnamed: 0,Country,University,QS Citations per Paper,QS Employer Reputation,QS Academic Reputation
0,0,United States,massachusetts institute of technology,93.7,98.2,91.0
1,1,United States,carnegie mellon university,95.3,82.3,100.0
2,2,United States,stanford university,99.9,96.3,88.5
3,3,United States,university of california berkeley,100.0,90.0,85.4
4,4,United Kingdom,university of oxford,94.2,96.7,82.1
...,...,...,...,...,...,...
681,681,Brazil,pontifcia universidade catlica do rio grande d...,61.2,52.5,41.3
682,682,Greece,technical university of crete,65.1,45.1,44.0
683,683,Austria,university of klagenfurt,70.9,37.7,46.0
684,684,France,universit lumire lyon 2,74.2,37.0,42.2


In [5]:
# THE uni name cleaning
df_the['University'] = df_the['University'].str.lower().replace('\s\(.*\)', '', regex=True)
df_the['University'] = df_the['University'].str.lower().replace('[^a-z0-9 ]', '', regex=True)
df_the

,Unnamed: 0,University,Country,Citations,Research,Teaching
0,0,university of oxford,United Kingdom,99.9,92.8,92.4
1,1,massachusetts institute of technology,United States,99.0,97.1,89.2
2,2,stanford university,United States,100.0,96.3,90.6
3,3,eth zurich,Switzerland,98.8,95.7,84.4
4,4,carnegie mellon university,United States,99.6,93.4,87.7
...,...,...,...,...,...,...
969,969,yamaguchi university,Japan,12.1,8.8,13.1
970,970,yanshan university,China,20.2,8.9,12.4
971,971,yokohama national university,Japan,20.6,19.3,17.0
972,972,yuan ze university,Taiwan,28.3,10.7,14.1


In [6]:
# SH uni name cleaning
df_shanghai['University'] = df_shanghai['University'].str.lower().replace('\s\(.*\)', '', regex=True)
df_shanghai['University'] = df_shanghai['University'].str.lower().replace('[^a-z0-9 ]', '', regex=True)
df_shanghai

,Unnamed: 0,University,CNCI,TOP
0,0,massachusetts institute of technology,81.1,100.0
1,1,stanford university,87.2,85.5
2,2,university of california berkeley,83.5,82.9
3,3,carnegie mellon university,73.7,99.8
4,4,tsinghua university,74.7,81.3
...,...,...,...,...
495,495,virginia commonwealth university,72.9,7.6
496,496,vrije universiteit brussel,68.1,11.0
497,497,wageningen university research,81.1,4.0
498,498,zhejiang normal university,83.0,6.9


In [7]:
from fuzzywuzzy import fuzz, process

# Define a function to find the best match for a given university name
def find_best_match(name, choices):
    best_match, score = process.extractOne(name, choices)
    if score >= 90:  # Adjust the threshold as needed
        return best_match
    else:
        return name

# Get unique university names from both dataframes
qs_universities = df_qs['University'].unique()
the_universities = df_the['University'].unique()

# Create a master list of all unique university names
all_universities = list(set(qs_universities))


# Standardize university names in df_the
df_the['New University'] = df_the.apply(lambda x: find_best_match(x['University'], all_universities), axis=1)

# Merge the dataframes based on standardized university names
merged_df = pd.merge(df_qs, df_the, on=['University', 'Country'], how='outer')

# Select the desired columns
merged_df = merged_df[['University', 'Country', 'QS Citations per Paper', 'QS Academic Reputation','QS Employer Reputation',
                       'Citations', 'Research', 'Teaching']]

# Display the merged dataframe
merged_df

,University,Country,QS Citations per Paper,QS Academic Reputation,QS Employer Reputation,Citations,Research,Teaching
0,massachusetts institute of technology,United States,93.7,91.0,98.2,99.0,97.1,89.2
1,carnegie mellon university,United States,95.3,100.0,82.3,99.6,93.4,87.7
2,stanford university,United States,99.9,88.5,96.3,100.0,96.3,90.6
3,university of california berkeley,United States,100.0,85.4,90.0,100.0,90.7,86.9
4,university of oxford,United Kingdom,94.2,82.1,96.7,99.9,92.8,92.4
...,...,...,...,...,...,...,...,...
1328,yamaguchi university,Japan,NaN,NaN,NaN,12.1,8.8,13.1
1329,yanshan university,China,NaN,NaN,NaN,20.2,8.9,12.4
1330,yokohama national university,Japan,NaN,NaN,NaN,20.6,19.3,17.0
1331,yuan ze university,Taiwan,NaN,NaN,NaN,28.3,10.7,14.1


In [8]:
# Get unique university names from both dataframes
shanghai_universities = df_shanghai['University'].unique()
merged_universities = merged_df['University'].unique()

# Create a master list of all unique university names
all_universities = list(set(merged_universities))

# Standardize university names in df_the
df_shanghai['new University'] = df_shanghai.apply(lambda x: find_best_match(x['University'], all_universities), axis=1)

# Merge the dataframes based on standardized university names
new_merged_df = pd.merge(merged_df, df_shanghai, on=['University'], how='outer')

# # Select the desired columns
new_merged_df = new_merged_df[['University', 'Country', 'QS Citations per Paper', 'QS Academic Reputation','QS Employer Reputation',
                       'Citations', 'Research', 'Teaching', 'CNCI', 'TOP']]

# Display the merged dataframe
new_merged_df

,University,Country,QS Citations per Paper,QS Academic Reputation,QS Employer Reputation,Citations,Research,Teaching,CNCI,TOP
0,massachusetts institute of technology,United States,93.7,91.0,98.2,99.0,97.1,89.2,81.1,100.0
1,carnegie mellon university,United States,95.3,100.0,82.3,99.6,93.4,87.7,73.7,99.8
2,stanford university,United States,99.9,88.5,96.3,100.0,96.3,90.6,87.2,85.5
3,university of california berkeley,United States,100.0,85.4,90.0,100.0,90.7,86.9,83.5,82.9
4,university of oxford,United Kingdom,94.2,82.1,96.7,99.9,92.8,92.4,72.2,63.9
...,...,...,...,...,...,...,...,...,...,...
1461,university of missouri columbia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.8,10.5
1462,university of paris,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56.7,23.1
1463,university of shanghai for science and technology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.1,6.9
1464,university of tennessee knoxville,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.4,14.5


In [9]:
# drop duplicates
new_merged_df.drop_duplicates(inplace=True)
new_merged_df

,University,Country,QS Citations per Paper,QS Academic Reputation,QS Employer Reputation,Citations,Research,Teaching,CNCI,TOP
0,massachusetts institute of technology,United States,93.7,91.0,98.2,99.0,97.1,89.2,81.1,100.0
1,carnegie mellon university,United States,95.3,100.0,82.3,99.6,93.4,87.7,73.7,99.8
2,stanford university,United States,99.9,88.5,96.3,100.0,96.3,90.6,87.2,85.5
3,university of california berkeley,United States,100.0,85.4,90.0,100.0,90.7,86.9,83.5,82.9
4,university of oxford,United Kingdom,94.2,82.1,96.7,99.9,92.8,92.4,72.2,63.9
...,...,...,...,...,...,...,...,...,...,...
1461,university of missouri columbia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.8,10.5
1462,university of paris,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56.7,23.1
1463,university of shanghai for science and technology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.1,6.9
1464,university of tennessee knoxville,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.4,14.5


In [10]:
# Fill NaN values with 0 
country_column = new_merged_df['Country']
new_merged_df.fillna(0, inplace=True)
new_merged_df[['University', 'Country', 'QS Citations per Paper', "Citations", 'CNCI', 'QS Academic Reputation',
              'QS Employer Reputation', 'Research', 'Teaching', 'TOP']]
new_merged_df

,University,Country,QS Citations per Paper,QS Academic Reputation,QS Employer Reputation,Citations,Research,Teaching,CNCI,TOP
0,massachusetts institute of technology,United States,93.7,91.0,98.2,99.0,97.1,89.2,81.1,100.0
1,carnegie mellon university,United States,95.3,100.0,82.3,99.6,93.4,87.7,73.7,99.8
2,stanford university,United States,99.9,88.5,96.3,100.0,96.3,90.6,87.2,85.5
3,university of california berkeley,United States,100.0,85.4,90.0,100.0,90.7,86.9,83.5,82.9
4,university of oxford,United Kingdom,94.2,82.1,96.7,99.9,92.8,92.4,72.2,63.9
...,...,...,...,...,...,...,...,...,...,...
1461,university of missouri columbia,0,0.0,0.0,0.0,0.0,0.0,0.0,73.8,10.5
1462,university of paris,0,0.0,0.0,0.0,0.0,0.0,0.0,56.7,23.1
1463,university of shanghai for science and technology,0,0.0,0.0,0.0,0.0,0.0,0.0,71.1,6.9
1464,university of tennessee knoxville,0,0.0,0.0,0.0,0.0,0.0,0.0,63.4,14.5


In [11]:
# ensure scores are of type int
new_merged_df['QS Citations per Paper'] = pd.to_numeric(new_merged_df['QS Citations per Paper'], errors='coerce')
new_merged_df['Citations'] = pd.to_numeric(new_merged_df['Citations'], errors='coerce')
new_merged_df['CNCI'] = pd.to_numeric(new_merged_df['CNCI'], errors='coerce')

# Calculate the "final citation score" for each row based on the specified logic
def calculate_final_score_citations(row):
    qs_citations_per_paper = row['QS Citations per Paper']
    citations = row['Citations']
    cnci = row['CNCI']

    if qs_citations_per_paper != 0 and citations != 0 and cnci != 0:
        final_score = (((qs_citations_per_paper + citations + cnci) / 3) / 100) * 0.8 + 0.2
    elif (qs_citations_per_paper != 0 and citations != 0) or (qs_citations_per_paper != 0 and cnci != 0) or (citations != 0 and cnci != 0):
        final_score = (((qs_citations_per_paper + citations + cnci) / 2) / 100) * 0.8 + 0.1
    elif qs_citations_per_paper != 0 or citations != 0 or cnci != 0:
        final_score = (max(qs_citations_per_paper, citations, cnci) / 100) * 0.8
    else:
        final_score = 0

    return (final_score*100)

# Apply the calculation to each row and create a new 'final citation score' column
new_merged_df['final citation score'] = new_merged_df.apply(calculate_final_score_citations, axis=1)

# Display the updated DataFrame
new_merged_df

,University,Country,QS Citations per Paper,QS Academic Reputation,QS Employer Reputation,Citations,Research,Teaching,CNCI,TOP,final citation score
0,massachusetts institute of technology,United States,93.7,91.0,98.2,99.0,97.1,89.2,81.1,100.0,93.013333
1,carnegie mellon university,United States,95.3,100.0,82.3,99.6,93.4,87.7,73.7,99.8,91.626667
2,stanford university,United States,99.9,88.5,96.3,100.0,96.3,90.6,87.2,85.5,96.560000
3,university of california berkeley,United States,100.0,85.4,90.0,100.0,90.7,86.9,83.5,82.9,95.600000
4,university of oxford,United Kingdom,94.2,82.1,96.7,99.9,92.8,92.4,72.2,63.9,91.013333
...,...,...,...,...,...,...,...,...,...,...,...
1461,university of missouri columbia,0,0.0,0.0,0.0,0.0,0.0,0.0,73.8,10.5,59.040000
1462,university of paris,0,0.0,0.0,0.0,0.0,0.0,0.0,56.7,23.1,45.360000
1463,university of shanghai for science and technology,0,0.0,0.0,0.0,0.0,0.0,0.0,71.1,6.9,56.880000
1464,university of tennessee knoxville,0,0.0,0.0,0.0,0.0,0.0,0.0,63.4,14.5,50.720000


In [12]:
#  Calculate the "final reputation score" for each row based on the specified logic
def calculate_final_score_rep(row):
    cols = ['TOP', 'QS Academic Reputation', 'QS Employer Reputation', 'Research', 'Teaching']
    non_zero_cols = [col for col in cols if row[col] != 0]
    non_zero_count = len(non_zero_cols)

    # Convert relevant columns to numeric
    numeric_cols = row[non_zero_cols].apply(pd.to_numeric, errors='coerce')

    if non_zero_count == 5:
        final_score = (((numeric_cols.sum() / 5) / 100) * 0.8) + 0.2
    elif non_zero_count == 4:
        final_score = (((numeric_cols.sum() / 4) / 100) * 0.8) + 0.15
    elif non_zero_count == 3:
        final_score = (((numeric_cols.sum() / 3) / 100) * 0.8) + 0.1
    elif non_zero_count == 2:
        final_score = (((numeric_cols.sum() / 2) / 100) * 0.8) + 0.05
    elif non_zero_count == 1:
        final_score = (numeric_cols[non_zero_cols[0]] / 100) * 0.8
    else:
        final_score = 0

    return (final_score * 100)

# Apply the calculation to each row and create a new 'final reputation score' column
new_merged_df['final reputation score'] = new_merged_df.apply(calculate_final_score_rep, axis=1)

# Display the updated DataFrame
new_merged_df

,University,Country,QS Citations per Paper,QS Academic Reputation,QS Employer Reputation,Citations,Research,Teaching,CNCI,TOP,final citation score,final reputation score
0,massachusetts institute of technology,United States,93.7,91.0,98.2,99.0,97.1,89.2,81.1,100.0,93.013333,96.080
1,carnegie mellon university,United States,95.3,100.0,82.3,99.6,93.4,87.7,73.7,99.8,91.626667,94.112
2,stanford university,United States,99.9,88.5,96.3,100.0,96.3,90.6,87.2,85.5,96.560000,93.152
3,university of california berkeley,United States,100.0,85.4,90.0,100.0,90.7,86.9,83.5,82.9,95.600000,89.744
4,university of oxford,United Kingdom,94.2,82.1,96.7,99.9,92.8,92.4,72.2,63.9,91.013333,88.464
...,...,...,...,...,...,...,...,...,...,...,...,...
1461,university of missouri columbia,0,0.0,0.0,0.0,0.0,0.0,0.0,73.8,10.5,59.040000,8.400
1462,university of paris,0,0.0,0.0,0.0,0.0,0.0,0.0,56.7,23.1,45.360000,18.480
1463,university of shanghai for science and technology,0,0.0,0.0,0.0,0.0,0.0,0.0,71.1,6.9,56.880000,5.520
1464,university of tennessee knoxville,0,0.0,0.0,0.0,0.0,0.0,0.0,63.4,14.5,50.720000,11.600


In [13]:
# Calculate overall score based on weightage for citations and reputation
new_merged_df['overall score'] = 0.2 * new_merged_df['final citation score'] + 0.8 * new_merged_df['final reputation score']
new_merged_df.sort_values(by='overall score', ascending=False, inplace=True)
new_merged_df.reset_index(drop=True, inplace=True)
new_merged_df

,University,Country,QS Citations per Paper,QS Academic Reputation,QS Employer Reputation,Citations,Research,Teaching,CNCI,TOP,final citation score,final reputation score,overall score
0,massachusetts institute of technology,United States,93.7,91.0,98.2,99.0,97.1,89.2,81.1,100.0,93.013333,96.080,95.466667
1,stanford university,United States,99.9,88.5,96.3,100.0,96.3,90.6,87.2,85.5,96.560000,93.152,93.833600
2,carnegie mellon university,United States,95.3,100.0,82.3,99.6,93.4,87.7,73.7,99.8,91.626667,94.112,93.614933
3,university of california berkeley,United States,100.0,85.4,90.0,100.0,90.7,86.9,83.5,82.9,95.600000,89.744,90.915200
4,university of oxford,United Kingdom,94.2,82.1,96.7,99.9,92.8,92.4,72.2,63.9,91.013333,88.464,88.973867
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1372,communication university of china,China,0.0,0.0,0.0,6.7,8.8,14.3,0.0,0.0,5.360000,14.240,12.464000
1373,mirea russian technological university,Russian Federation,0.0,0.0,0.0,11.8,7.9,10.9,0.0,0.0,9.440000,12.520,11.904000
1374,kansai university,Japan,0.0,0.0,0.0,3.3,8.1,14.0,0.0,0.0,2.640000,13.840,11.600000
1375,metropolitan autonomous university,Mexico,0.0,0.0,0.0,7.2,8.4,11.3,0.0,0.0,5.760000,12.880,11.456000


In [14]:
# Download to csv
new_merged_df.to_csv('FINAL_ranking_list.csv')